In [73]:
import os
import pandas as pd
import git
import numpy as np



# folder = "C:\\Users\\Van\\Desktop\\vancanwin\\pgap3cdg\\01.20.23\\"

def csv_to_excel(folder,filename):
    filetype = filename.split(".")[-1]
    if "csv" in filetype:
        df = pd.read_csv(folder+filename, sep='delimiter')#,on_bad_lines='skip')
        df = df.iloc[:,0].str.split(",",expand=True)
        filename = filename.split(".")[0]+".xlsx"
        df.to_excel(folder+filename)
        #     if "csv" in filetype:
        # df = pd.read_csv(folder+filename, sep='delimiter',on_bad_lines='skip')
        # df = df.iloc[:,0].str.split(",",expand=True)
    
def normalize_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = filename.split(".")[-1]
    df = pd.DataFrame([])

    
    df = pd.read_csv(folder+filename, on_bad_lines='skip', encoding = "utf-8",sep=',').to_numeric(errors='ignore')
    col_num = df.shape[1]
    [index_start, index_end] = [0,16]
    column_index_end = 24
    df_results_index = 0
    for column in range(col_num):
        print("column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False)].index
            column_header = column
            column_index = column+1
            print("line 25 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            pass
    for i, index in enumerate(df_results_index):
        # dateset_name = df.iloc[index,0]
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:
                print("checking 36",df_results)
                df_results = df_results.fillna(0)
                df_results = df_results.astype(int)
                df_results.columns = new_header
                df_median_neg_controls = int(df_results.iloc[:,0:2].stack().median())
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range
                # print('checking',  df.iloc[index+index_start:index+index_end,column_index-1:column_index+column_index_end])
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_norm
                print("processed...",i)
            else:
                print("skip data")
                pass
        except Exception as e: 
            print(e)
            pass

    # df = df.iloc[:,column_index:column_index+column_index_end]
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_normalized."+filename.split(".")[-1])
    return(df)

def zscore_plate384(folder,filename):
    '''Enter file to read plate and normalize data'''
    print(folder+filename)
    filetype = "csv"
    filename = filename.split(".")[0]
    print("Filename z-score",folder+filename, filetype)

    df = pd.DataFrame([])
    df = pd.read_csv(folder+"\\completed\\"+filename+"_normalized."+filetype, encoding = "utf-8",sep=',')
    col_num = df.shape[1]
    index_start = 0
    index_end = 16
    column_index_end = 24
  
    for column in range(col_num):
        print("z-score column number", column, type(column))
        try:
            df_results_index = df[df.iloc[:,column].str.contains('A', na=False)].index
            column_index = column+1
            print("line 95 matching?",df_results_index)
            break
        except Exception as e:
            print(e)
            pass
    for i, index in enumerate(df_results_index):
        new_header = df.iloc[index-1,column_index:column_index+column_index_end]
        df_results = df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] #row,col
        count_na = df_results.isna().sum().sum()
        count_values = df_results.count().sum()

        print("zscore NA count:", count_na, "Values count:", count_values, "done")
        try:
            if  count_na < 100 and count_values > 100:

                df_results = df_results.fillna(0)
                df_results = df_results.astype(int)

                df_results.columns = new_header
                df_min = df_results.stack().min()
                df_max = df_results.stack().max()
                df_range = df_max-df_min
                df_results_norm = (df_results - df_min)/df_range

                df_mean = df_results_norm.stack().mean()
                df_std = df_results_norm.stack().std()
                df_results_zscore = (df_results_norm - df_mean)/df_std
                df.iloc[index+index_start:index+index_end,column_index:column_index+column_index_end] = df_results_norm
                print("zscore processed...",i)
            else:
                print("zscore skip data")
                pass
        except Exception as e: 
            print(e)
            pass
    # df = df.iloc[:,column_index:column_index+column_index_end]
    df.to_csv(folder+"\\completed\\"+filename.split(".")[0]+"_zscore.csv")

    return(df)

repo = git.Repo('.', search_parent_directories=True)
folder = repo.working_tree_dir +"\\yeast-data\\"

plate_list = os.listdir(folder)
print(plate_list,folder)

for file in plate_list:
    filetype = file.split(".")[-1]
    if filetype == "csv":
        print("Filename:", file)
        normalize_plate384(folder,file)
        zscore_plate384(folder,file)
       
    else:
        print(filetype)
        pass

   

['.~lock.20230322_Surf1_plate1to5.csv#', 'archive', 'completed', 'plate-data-20230322_Surf1_plate1to5.csv', 'plate-data-20230322_Surf1_plate6to12.csv', 'plate-data-20230330_SURF1(batch 2).csv', 'plate-data-20230412_SURF1_Last16plates.csv'] C:\Users\Van\Desktop\vancanwin\yeast-data\
csv#
archive
completed
Filename: plate-data-20230322_Surf1_plate1to5.csv
C:\Users\Van\Desktop\vancanwin\yeast-data\plate-data-20230322_Surf1_plate1to5.csv


AttributeError: 'DataFrame' object has no attribute 'to_numeric'